# Using `OpenAI`'s GPT model

In [ ]:
!pip install -r requirements.txt --quiet

## Generate API key
Go to: https://auth.openai.com/log-in and generate an accout and set up and API key.

This can be found under:

1. Dashboard 
2. API keys(https://platform.openai.com/api-keys)

In [ ]:
import os
from getpass import getpass
import openai

os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## Code generation with GPT

In [ ]:
USER_PROMPT = """
Write a Python function that does a leet coding 
Recursive Binary Search.
The function should take a sorted list of integers 
and a target integer as input, 
and return the index of the target integer in the 
list if it exists, or -1 if it does not.
The function should be implemented using recursion.
The function should be named `recursive_binary_search` 
and should have the following signature:

```python   
def recursive_binary_search(arr: list[int], 
                            target: int) -> int:
```
"""

In [ ]:
SYSTEM_PROMPT = """
You are a helpful assistant that generates Python code 
based on the provided prompt.
"""

## Create Model

https://platform.openai.com/docs/models

In [ ]:
MODEL = "gpt-4o"

In [ ]:
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT}
    ],
    max_tokens = 2048,
    temperature = 0.3
)

In [ ]:
generated_code = response.choices[0].message.content
print(generated_code)

## Extract Python code

In [ ]:
import re
raw_output = response.choices[0].message.content
match = re.search(r"```python\s+(.*?)```", 
                  raw_output, re.DOTALL)
if match:
    python_code = match.group(1).strip()
    print(python_code)
else:
    print("No Python code block found.")

## Display Python code

In [ ]:
from IPython.display import display, Markdown
display(Markdown(f"```python\n{python_code}\n```"))

## Execute Python code

In [ ]:
exec(python_code)

In [ ]:
sorted_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
target = 5
result = recursive_binary_search(sorted_list, target)
print(f"The TARGET specified: {target} is at INDEX: {result} due to Python's 🐍 zero-based indexing.")  

# Summarisation and Translation with OpenAI

In [ ]:
SYSTEM_PROMPT = """
You are a helpful assistant that summarises long text and 
produces a concise, but informative summary. Please explain your response in 
a way that is easy to understand for a general audience.
"""

In [ ]:
USER_PROMPT = """
Machine ethics
Main articles: Machine ethics and AI alignment
Machine ethics (or machine morality) is the field of research concerned 
with designing Artificial Moral Agents (AMAs), 
robots or artificially intelligent computers that behave morally or as though moral.[4][5][6][7] 
To account for the nature of these agents, it has been suggested to consider certain philosophical ideas, 
like the standard characterizations of agency, rational agency, moral agency,
 and artificial agency, which are related to the concept of AMAs.[8]
There are discussions on creating tests to see if an AI is capable of making ethical decisions. 
Alan Winfield concludes that the Turing test is flawed and the requirement for an 
AI to pass the test is too low.[9] A proposed alternative test is one called the Ethical Turing Test, 
which would improve on the current test by having multiple judges decide if the 
AI's decision is ethical or unethical.[9] Neuromorphic AI could be one way to create morally capable robots, 
as it aims to process information similarly to humans, nonlinearly and with millions of 
interconnected artificial neurons.[10] Similarly, whole-brain emulation (scanning a brain and simulating it on digital hardware) 
could also in principle lead to human-like robots, thus capable of moral actions.[11] 
And large language models are capable of approximating human moral judgments.[12] Inevitably, 
this raises the question of the environment in which such robots would learn about the world and 
whose morality they would inherit – or if they end up developing human 'weaknesses' as well: selfishness, 
pro-survival attitudes, inconsistency, scale insensitivity, etc.
"""

In [ ]:
def get_openai_response(user_prompt, system_prompt, 
                        model="gpt-4o", **kwargs):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        **kwargs
    )
    return response.choices[0].message.content, response

In [ ]:
response, _ = get_openai_response(user_prompt=USER_PROMPT, 
                    system_prompt=SYSTEM_PROMPT, 
                    model=MODEL, 
                    max_tokens=2048, 
                    temperature=0.3)

In [ ]:
print(response)

In [ ]:
len(response)


In [ ]:
len(USER_PROMPT)

## Translation step

In [ ]:
SYSTEM_PROMPT = """
Your job is to translate the provided source text into German.
Please ensure that the translation is accurate and maintains the original meaning.
"""

In [ ]:
translated_resp, _ = get_openai_response(
    user_prompt=response, 
    system_prompt=SYSTEM_PROMPT, 
    model=MODEL, 
    max_tokens=2048, 
    temperature=0.3)

In [ ]:
translated_resp

## Back translate with Google Translate

In [ ]:
from googletrans import Translator
async def translate_text(text_to_translate, 
                         dest='en', src='de', **kwargs):
    """Asynchronously translates text using Google Translate."""
    async with Translator() as translator:
         """Async context manager for the translator."""
         result = await translator.translate(text_to_translate, 
                                             dest=dest, 
                                             src=src, **kwargs)
         return result
google_translate_res = await translate_text(text_to_translate=translated_resp)

In [ ]:
google_translate_res.text